## Comp3419 Assignment_1a
## 500177868

In [1]:
## import library
import sys
import os
import cv2
import numpy as np
import math
from matplotlib import pyplot as plt
%matplotlib inline

#### Check the path and open video

In [2]:
## path name
path_to_video = 'monkey.avi'
frame_save_path = './frames/'
composite_save_path = './composite/'
new_composite_save_path = './new_composite/'

In [3]:
## create directory
def create_dir_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [4]:
## open video and check whether dir is exist and create if not
cap = cv2.VideoCapture(path_to_video)
create_dir_if_not_exists(frame_save_path)
create_dir_if_not_exists(composite_save_path)
create_dir_if_not_exists(new_composite_save_path)
if not cap.isOpened():
    print('{} not opened'.format(path_to_video))
    sys.exit(1)
# Use cv2 to fetch three important variables
time_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)

print(time_length)
print(frame_height)
print(frame_width)

752
576.0
720.0


#### Extract Frames from Monkey video

In [5]:
frame_counter = 0    
while(1):
    return_flag, frame = cap.read()  
    if not return_flag:
        print('Video Reach End')
        break
    cv2.imwrite(frame_save_path + 'frame%d.tif' % frame_counter, frame)
    frame_counter += 1
    if cv2.waitKey(30) & 0xff == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Video Reach End


#### Motion Estimation

In [6]:
def arrowdraw(img, x1, y1, x2, y2):
    radians = math.atan2(x1-x2, y2-y1)
    x11 = 0
    y11 = 0
    x12 = -10
    y12 = -10

    u11 = 0
    v11 = 0
    u12 = 10
    v12 = -10
    
    x11_ = x11*math.cos(radians) - y11*math.sin(radians) + x2
    y11_ = x11*math.sin(radians) + y11*math.cos(radians) + y2

    x12_ = x12 * math.cos(radians) - y12 * math.sin(radians) + x2
    y12_ = x12 * math.sin(radians) + y12 * math.cos(radians) + y2
    
    u11_ = u11 * math.cos(radians) - v11 * math.sin(radians) + x2
    v11_ = u11 * math.sin(radians) + v11 * math.cos(radians) + y2

    u12_ = u12 * math.cos(radians) - v12 * math.sin(radians) + x2
    v12_ = u12 * math.sin(radians) + v12 * math.cos(radians) + y2
    img = cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 5)
    img = cv2.line(img, (int(x11_), int(y11_)), (int(x12_), int(y12_)), 
    (255, 0, 0), 5)
    img = cv2.line(img, (int(u11_), int(v11_)), (int(u12_), int(v12_)), 
    (255, 0, 0), 5)
    
    return img

In [7]:
## initial value
k = 4
search_area = 10
Tmin = 0
Tmax = 10
frame_counter = 0
## Iterate all frames
while(True):
    ## read frame with current ad next one
    current_frame = cv2.imread(frame_save_path + 'frame%d.tif' % frame_counter)
    if current_frame is None:
        print('Reach end')
        break
    next_frame = cv2.imread(frame_save_path + 'frame%d.tif' % (frame_counter + 1))
    ## define grid blocks for current frame
    current_grid = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
    next_grid = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)
    ## loop all block in current frame
    for y in range(5, int(frame_height), 2 * k + 1):
        for x in range(5, int(frame_width), 2 * k + 1):
            ## difine the min of SSD
            min_SSD = 526
            ## best match value
            best_x = 0
            best_y = 0
            ## if pixel is blue then to the next block
            if current_frame[y][x][0] >= 100 or next_frame[y][x][0] >= 100:
                continue
            # Search for the best matching block in the search range
            for search_y in range(y - search_area, y + search_area + 1):
                ## if out of range then go to nexy
                if search_y < 0 or search_y + 2 * k >= frame_height: 
                    continue
                for search_x in range(x - search_area, x + search_area + 1):
                    if search_x < 0 or search_x + 2 * k >= frame_width:
                        continue
                    ## calculate ssd
                    ssd = np.sqrt(np.sum((current_grid[y-k:y+k,x-k:x+k]-next_grid[search_y-k:search_y+k,search_x-k:search_x+k])**2))
                    ## select minimum ssd
                    if ssd < min_SSD:
                        min_SSD = ssd
                        best_x = search_x
                        best_y = search_y
            ## if ssd is in range (Tmin,Tmax), then fraw arrow on it
            if min_SSD < Tmax and  min_SSD > Tmin:
                new_img = arrowdraw(current_frame, x, y, best_x, best_y)
                cv2.imwrite('composite/composite%d.tif' % frame_counter, new_img)
    frame_counter += 1

Reach end


[ WARN:0@1013.218] global loadsave.cpp:244 findDecoder imread_('./frames/frame752.tif'): can't open/read file: check file path/integrity


#### Convert extracted frame to result video

In [11]:
frame_load_path = 'composite/'
path_to_output_video = 'output.avi'
video = cv2.imread('composite/composite1.tif')
frame_height = video.shape[0]
frame_width = video.shape[1]
out = cv2.VideoWriter(path_to_output_video, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (int(frame_width), int(frame_height)))
frame_counter = 0
while(1):
    img = cv2.imread(frame_load_path + 'composite%d.tif' % frame_counter)
    if frame_counter == 752:
        print('No more frames to be loaded')
        break
    out.write(img)
    frame_counter += 1
out.release()
cv2.destroyAllWindows()

No more frames to be loaded


#### If change the T range then we could have different arrow

In [12]:
## initial value
k = 4
search_area = 10
Tmin = 0
Tmax = 20
frame_counter = 0
## Iterate all frames
while(True):
    ## read frame with current ad next one
    current_frame = cv2.imread(frame_save_path + 'frame%d.tif' % frame_counter)
    next_frame = cv2.imread(frame_save_path + 'frame%d.tif' % (frame_counter + 1))
    ## check whether no more frames
    if (current_frame is None) or (next_frame is None):
        print('Reach end')
        break
    ## define grid blocks for current frame
    current_grid = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
    next_grid = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)
    ## loop all block in current frame
    for y in range(5, int(frame_height), 2 * k + 1):
        for x in range(5, int(frame_width), 2 * k + 1):
            ## difine the min of SSD
            min_SSD = 526
            ## best match value
            best_x = 0
            best_y = 0
            ## if pixel is blue then to the next block
            if current_frame[y][x][0] >= 100 or next_frame[y][x][0] >= 100:
                continue
            # Search for the best matching block in the search range
            for search_y in range(y - search_area, y + search_area + 1):
                ## if out of range then go to nexy
                if search_y < 0 or search_y + 2 * k >= frame_height: 
                    continue
                for search_x in range(x - search_area, x + search_area + 1):
                    if search_x < 0 or search_x + 2 * k >= frame_width:
                        continue
                    ## calculate ssd
                    ssd = np.sqrt(np.sum((current_grid[y-k:y+k,x-k:x+k]-next_grid[search_y-k:search_y+k,search_x-k:search_x+k])**2))
                    ## select minimum ssd
                    if ssd < min_SSD:
                        min_SSD = ssd
                        best_x = search_x
                        best_y = search_y
            ## if ssd is in range (Tmin,Tmax), then fraw arrow on it
            if min_SSD < Tmax and  min_SSD > Tmin:
                new_img = arrowdraw(current_frame, x, y, best_x, best_y)
                cv2.imwrite('new_composite/composite%d.tif' % frame_counter, new_img)
    frame_counter += 1

Reach end


In [13]:
frame_load_path = 'new_composite/'
path_to_output_video = 'new_output.avi'
video = cv2.imread('new_composite/composite1.tif')
frame_height = video.shape[0]
frame_width = video.shape[1]
out = cv2.VideoWriter(path_to_output_video, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (int(frame_width), int(frame_height)))
frame_counter = 0
while(1):
    img = cv2.imread(frame_load_path + 'composite%d.tif' % frame_counter)
    if frame_counter == 752:
        print('No more frames to be loaded')
        break
    out.write(img)
    frame_counter += 1
out.release()
cv2.destroyAllWindows()

No more frames to be loaded
